In [2]:
'''
pretty much deprecated, see testing_real_imgs.py
'''
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import json
import cv2
import matplotlib.image
from glob import glob
import torch
from model_factory import get_model
import argparse
from model_factory import UNet, UNet_Single
from hsqc_dataset import *
from tqdm import tqdm

"""configurations"""
device = torch.device("cuda:0")
json_version = "bitmap"
config_path = f"/root/autoencoder_denoiser/configs_{json_version}"
version = "bitmap"
exp_dir = f'/root/autoencoder_denoiser/exps/results_{version}'

class Test():
    def __init__(self, model_name, 
                    config_path = config_path,
                    exp_dir = exp_dir,
                    model_path = None) -> None:
        self.dilation = False
        self.resize = True
        self.config = None
        name = model_name
        self.name = name

        f = open(f'{config_path}/'+ name + '.json')
        self.config = json.load(f)
        if model_path:
            state_dict = torch.load(model_path)
        else:
            experiment_path = f"{exp_dir}/{name}/" 
            print('load from: ', os.path.join(experiment_path, 'latest_model.pt'))
            state_dict = torch.load(os.path.join(experiment_path, 'latest_model.pt'))
        model = get_model(self.config)
        # try:
        model.load_state_dict(state_dict['model'])
        model = torch.nn.DataParallel(model)
        # except:
        #     model = torch.nn.DataParallel(model)
        #     model.load_state_dict(state_dict['model'])
            
        model.to(device)
        model.eval()
        self.model = model

        
# my_model_test = Test("t1_03")
# paper_1d_test = Test("paper_1d")




In [15]:
# dann_test = Test('adv')
# cdan_test = Test('cdan')
# cdan_e_test=  Test('cdan_e')
# paper1d_test = Test("paper1d")
# match_hist_baseline_test = Test("match_hist")
# need_display = True
# one_d_test= Test('paper_1d', config_path='/root/autoencoder_denoiser/configs')
# CE_loss = Test("CE_loss_old_dataset")

my_model_test = Test("match_hist")


load from:  /root/autoencoder_denoiser/exps/results_bitmap/adv/latest_model.pt
model: Adv_Unet
load from:  /root/autoencoder_denoiser/exps/results_bitmap/match_hist/latest_model.pt
model :UNet


In [5]:
# paper_1d_test = Test("paper_1d", model_path="/root/autoencoder_denoiser/previous_paper_denoise/exp_results/orig_config_with_norm_v4/best_model.pt")

In [7]:
my_model_test = Test("match_hist")

from utils import display_pics

# '''creating test dataloader of real noise'''
config = my_model_test.config
# batch = config["dataset"]['batch_size']
# shuffle=config["dataset"]['shuffle']
need_display = True
test_loader = DataLoader(RealNoiseDataset_Byeol(config), batch_size=2, shuffle=False, num_workers=1)
criterion = torch.nn.MSELoss(reduction="sum")


load from:  /root/autoencoder_denoiser/exps/results_bitmap/match_hist/latest_model.pt
model :UNet
using Byeol's real imgs: normal noise


In [9]:

clist = [(0,"darkblue"), (0.5,"white"), (1, "darkred")]
custom_HSQC_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("_",clist)
diff_clist = [(0,"green"), (0.5,"white"), (1, "red")]
custom_diff_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("_",diff_clist)


def compute_SNR(raw, noisy_img): 
    signal_position= torch.where(raw!=0)
    # noise_position= torch.where(raw==0)
    # prediction_error = torch.sum( torch.abs(raw-noisy_img))
 
    avg_signal = torch.sum( torch.abs(raw))/len(signal_position[0])
    noise_std =  torch.std(noisy_img - raw)
    return (avg_signal/noise_std).item()

def test(*model_tests):
    for model_test in model_tests:
        displayed=0
        display_num = 0
        loss = 0
        snr = 0
        plt.rcParams["figure.figsize"] = (20,10)
        with torch.no_grad():
            for iter, data in enumerate(tqdm(test_loader)):
                noise, raw = data
                if len(raw.shape)==3:   
                    raw, noise = raw.unsqueeze(1), noise.unsqueeze(1)
                raw, noise = raw.to(device).float(), noise.to(device).float()
                prediction = model_test.model.forward(noise)
                
                # find loss
                # prediction = prediction.type(torch.float32)
                ground_truth = raw
            
                # add adv loss !!!
                prediction = torch.clip(prediction,-1,1)

            # print(denoised_1.shape)
            # print(ground_truth.shape)
                loss += criterion(prediction,ground_truth )
                snr += compute_SNR(raw, prediction)
            
                if need_display and displayed<10:
                    noise_pic , prediction_pic, raw_pic = noise[1],prediction[1], raw[1]
                    
                
                    test_samples_path = f"/root/autoencoder_denoiser/testing_real_imgs_results/{model_test.name}/"
                    os.makedirs(test_samples_path, exist_ok=True)
                    save_path = (os.path.join(test_samples_path, f"sample_image{displayed}.png"))
                    
                    display_pics(noise_pic[0].cpu(), prediction_pic[0].cpu(), raw_pic[0].cpu(), save_path=save_path)
                    
                    displayed = displayed+1
            
                    
            loss /= len(test_loader.dataset)  
            snr /=   len(test_loader.dataset)
            print("test loader size:" , len(test_loader.dataset))
            print(f"loss of model: {model_test.name} is {loss}")
            print(f"snr of model: {model_test.name} is {snr}")

    
    

In [10]:
# test 1d :
state_dict = torch.load('/root/autoencoder_denoiser/previous_paper_denoise/exp_results/orig_config_with_norm_v4/best_model.pt')
specs = ( [64, 128, 1024],

        [ 512, 256, 64])


model = UNet_Single(1,1,False,1,channel_specs= specs)
model.load_state_dict(state_dict['model'])
model = torch.nn.DataParallel(model).to("cuda")

displayed=0
display_num = 0
loss = 0
snr = 0
plt.rcParams["figure.figsize"] = (20,10)

def normalize_perdiction(prediction):
    """normalizing the prediction result"""
    prediction = prediction.double()
            # print(prediction)
    prediction /= torch.max(torch.abs(prediction))
    return prediction

with torch.no_grad():
    for iter, data in enumerate(tqdm(test_loader)):
        noise, raw = data
        raw, noise = raw.to(device).float(), noise.to(device).float()
        
        prediction = torch.zeros(noise.shape).to("cuda")
        for i in range(len(prediction)):
            for j in range(len(prediction[i][0])):
                curr = noise[i][0][j]
                signs = torch.sign(curr)
                prediction[i][0][j] = normalize_perdiction(model.forward((curr))) * signs
                
        loss += criterion(prediction,raw )
        snr += compute_SNR(raw, prediction)
  

    
        if need_display and displayed<2:
                noise_pic , prediction_pic, raw_pic = noise[1],prediction[1], raw[1]
                
        test_samples_path = f"/root/autoencoder_denoiser/testing_real_imgs_results/paper_1d/"
        os.makedirs(test_samples_path, exist_ok=True)
        save_path = (os.path.join(test_samples_path, f"sample_image{displayed}.png"))
        
        display_pics(noise_pic[0].cpu(), prediction_pic[0].cpu(), raw_pic[0].cpu(), save_path=save_path)
        
        displayed = displayed+1
    
            
    loss /= len(test_loader.dataset)  
    snr /=  len(test_loader.dataset)
    print("test loader size:" , len(test_loader.dataset))
    print(f"loss of model: paper_1d is {loss}")
    print(f"snr of model: paper_1d is {snr}")

100%|██████████| 30/30 [01:33<00:00,  3.11s/it]

test loader size: 59
loss of model: paper_1d is 756.748779296875
snr of model: paper_1d is 0.7644067255117125


<Figure size 432x288 with 0 Axes>

In [32]:
a = torch.tensor([1,2,3]).double()
torch.where(a > 1.3,a ,0.)

tensor([0., 2., 3.], dtype=torch.float64)

In [ ]:
plt.rcParams["figure.figsize"] = (20,16)

# test(match_hist_baseline_test )
#



In [48]:
test( my_model_test)
# test( paper_1d_test)


100%|██████████| 30/30 [00:11<00:00,  2.61it/s]

test loader size: 59
loss of model: match_hist is 141.350830078125
snr of model: match_hist is 6.2938682184381


<Figure size 1440x720 with 0 Axes>

In [ ]:
'''testing thresholding'''
from hsqc_dataset import *

plt.rcParams["figure.figsize"] = (20,10)

test_loader = DataLoader(RealNoiseDataset_Byeol(dann_test.config,range_low=0,range_high=2, show_name=True),
                         batch_size=1, shuffle=False, num_workers=8)


def test_thresholding(test_loader, threshold_value = 0.4, dir_name_to_save = "resized_thresholding"):
    testing_result_dir = f'/root/autoencoder_denoiser/testing_real_imgs_results/{test_loader.dataset.data_folder_name}_{dir_name_to_save}/'
    os.makedirs(testing_result_dir, exist_ok= True)
    displayed=0
    display_num = 0
    loss = 0
    snr = 0
    snr_orig = 0
    with torch.no_grad():
        for iter, data in enumerate(tqdm(test_loader)):
            noise, raw,name = data
            name = "".join([chr(i) for i in name[0]])
            if len(raw.shape)==3:   
                raw, noise = raw.unsqueeze(1), noise.unsqueeze(1)
            prediction = torch.clone(noise)
            prediction[abs(prediction)<threshold_value]=0
        
        # find loss
        # prediction = prediction.type(torch.float32)
            ground_truth = raw
    
        # add adv loss !!!
            prediction = torch.clip(prediction,-1,1)

    # print(denoised_1.shape)
    # print(ground_truth.shape)
            loss += criterion(prediction,ground_truth )
            snr += compute_SNR(raw, prediction)
            snr_orig += compute_SNR(raw, noise)
    
        # if need_display and displayed<2:
            if False:
                noise_pic , prediction_pic, raw_pic = noise[0],prediction[0], raw[0]
            
            # print("?")
            # plt.clf()

                ax = plt.subplot(2, 2, 1)
                plt.tight_layout()
                ax.set_title('original',fontsize=18)
            # ax.axis('off')
                plt.imshow(raw_pic[0].cpu(),cmap=custom_HSQC_cmap, vmax=1, vmin=-1)
                plt.colorbar()
            
                ax = plt.subplot(2, 2, 2)
                plt.tight_layout()
                ax.set_title('noise',fontsize=18)
            # ax.axis('off')
                plt.imshow(noise_pic[0].cpu(),cmap=custom_HSQC_cmap, vmax=1, vmin=-1)
                plt.colorbar()
            
                ax = plt.subplot(2, 2, 3)
                plt.tight_layout()
                ax.set_title('predicted',fontsize=18)
            # ax.axis('off')
                plt.imshow(prediction_pic[0].cpu(),cmap=custom_HSQC_cmap, vmax=1, vmin=-1)
                plt.colorbar()
            
                ax = plt.subplot(2, 2, 4)
                plt.tight_layout()
                ax.set_title('difference', fontsize=18)
            # ax.axis('off')
            
            # difference = prediction_pic[0].cpu()-raw_pic[0].cpu()
            # difference = difference.float()/2 + 0.5
            # print(difference)
                difference = cv2.subtract(np.array(prediction_pic[0].cpu()), np.array(raw_pic[0].cpu()))
                plt.imshow(difference, cmap = custom_diff_cmap, vmax=1, vmin=-1)
            
                plt.colorbar()
                plt.savefig(f'/root/autoencoder_denoiser/testing_real_imgs_results_bitmaps/{dir_name_to_save}/result_{name}.png')
                plt.clf()
                plt.figure()

                displayed = displayed+1
            
        loss /= len(test_loader.dataset)  
        snr /=   len(test_loader.dataset)
        snr_orig /=   len(test_loader.dataset)

        print("test loader size:" , len(test_loader.dataset))
        print(f"loss of model: thresholding is {loss}")
        print(f"snr of model: thresholding is {snr}")





KeyError: 'super_noisy'

In [ ]:
test_thresholding(test_loader, threshold_value = 0.4 , dir_name_to_save = "resized_thresholding")


100%|██████████| 22/22 [00:02<00:00,  9.47it/s]

test loader size: 22
loss of model: thresholding is 5.9233598709106445
snr of model: thresholding is 89.25157668373801


In [ ]:
f = open(f'{config_path}/'+ 'adv' + '.json')
config = json.load(f)
config['dataset']['real_img_keep_size'] = True 
test_loader = DataLoader(RealNoiseDataset_Byeol(config,range_low=0,range_high=2, show_name=True),
                         batch_size=1, shuffle=False, num_workers=8)

# test_thresholding(test_loader, threshold_value = 0.4 , dir_name_to_save = "orig_size_thresholding")


In [ ]:
for data in test_loader:
    noise, raw,name = data
    break
    

In [ ]:
# chr(104)
print(noise.nelement())
print(noise.element_size())
# sys.getsizeof(noise)

a= raw.to_sparse()

print(a.nelement())
print(a.element_size())

a.values().size()

65630720
4
65630720
4


torch.Size([98132])

In [ ]:
# baseline_test = Test('baseline', config_path='/root/autoencoder_denoiser/configs_4.13_solved_iterator_bug',
                    #  exp_dir='/root/autoencoder_denoiser/exps/results_4.13_solved_iterator_bug')
# test(baseline_test)

In [ ]:
# # my_model_test = Test("t1_03")
# baseline_test = Test('paper_1d', config_path='/root/autoencoder_denoiser/configs',
#                      exp_dir='/root/autoencoder_denoiser/exps/results_4.13_solved_iterator_bug')
# test(baseline_test)
# # paper_1d_test = Test("paper_1d")

In [ ]:
# def loop_iterable(iterable):
#     while True:
#         yield from iterable
# a=  (loop_iterable([1,2,3,4,5]))


In [ ]:
# a = zip([1,2],3,4,5,6])

In [ ]:
# next(a)

In [ ]:
# import json, torch
# f = open('/root/autoencoder_denoiser/configs/'+ 't1_03' + '.json')
# config = json.load(f)

# state_dict = torch.load('/root/autoencoder_denoiser/results_new_SNR/t1_03/latest_model.pt')
# model = get_model(config)
# model = torch.nn.DataParallel(model)
# model.load_state_dict(state_dict['model'])

# torch.save(model.module.state_dict(), '/root/autoencoder_denoiser/results_new_SNR/t1_03/latest_model_weight.pt') 

In [ ]:
def tt(*nums):
    for n in nums:
        print(n**2)
        
tt(12,23)

144
529


In [ ]:
import sys, torch
a = torch.tensor([0,0,0,0,0,1,0,0,0,2])
a.to_sparse()
sys.getsizeof(a)


72